In [1]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   514k      0  0:00:01  0:00:01 --:--:--  514k


In [2]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [3]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [4]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

The dimensions of movies dataframe are: (9742, 3) 
The dimensions of ratings dataframe are: (100836, 4)


In [5]:
# Take a look at movies_df
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# Take a look at ratings_df
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [8]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [9]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [10]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

Is running on GPU: True
MatrixFactorization(
  (user_factors): Embedding(610, 8)
  (item_factors): Embedding(9724, 8)
)
user_factors.weight tensor([[0.0462, 0.0429, 0.0116,  ..., 0.0389, 0.0072, 0.0021],
        [0.0177, 0.0353, 0.0072,  ..., 0.0422, 0.0199, 0.0191],
        [0.0225, 0.0495, 0.0362,  ..., 0.0194, 0.0262, 0.0118],
        ...,
        [0.0484, 0.0067, 0.0364,  ..., 0.0227, 0.0166, 0.0128],
        [0.0380, 0.0427, 0.0493,  ..., 0.0445, 0.0484, 0.0061],
        [0.0079, 0.0301, 0.0071,  ..., 0.0396, 0.0405, 0.0201]])
item_factors.weight tensor([[0.0441, 0.0342, 0.0206,  ..., 0.0098, 0.0424, 0.0124],
        [0.0139, 0.0145, 0.0333,  ..., 0.0104, 0.0285, 0.0224],
        [0.0332, 0.0037, 0.0323,  ..., 0.0437, 0.0183, 0.0177],
        ...,
        [0.0319, 0.0294, 0.0036,  ..., 0.0340, 0.0475, 0.0277],
        [0.0186, 0.0069, 0.0204,  ..., 0.0147, 0.0295, 0.0077],
        [0.0316, 0.0134, 0.0148,  ..., 0.0207, 0.0139, 0.0078]])


In [11]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-11-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.062329396983694
iter #1 Loss: 4.738543045399758
iter #2 Loss: 2.4740186017479386
iter #3 Loss: 1.7206953552925042
iter #4 Loss: 1.345305378440068
iter #5 Loss: 1.1284330934286118
iter #6 Loss: 0.991446352776537
iter #7 Loss: 0.9001344662481153
iter #8 Loss: 0.837108101525585
iter #9 Loss: 0.7921220126188346
iter #10 Loss: 0.7591841287419275
iter #11 Loss: 0.7346255369295324
iter #12 Loss: 0.7160685511712496
iter #13 Loss: 0.701593720292682
iter #14 Loss: 0.6904197126097485
iter #15 Loss: 0.6818936536381692
iter #16 Loss: 0.6751768155978416
iter #17 Loss: 0.6693802533415973
iter #18 Loss: 0.6656439321370899
iter #19 Loss: 0.6627489358640565
iter #20 Loss: 0.6605599578564543
iter #21 Loss: 0.6587538196123796
iter #22 Loss: 0.6579971665960883
iter #23 Loss: 0.6564779752477776
iter #24 Loss: 0.6556920172069883
iter #25 Loss: 0.6548382657660445
iter #26 Loss: 0.6540415359254416
iter #27 Loss: 0.6531615158174243
iter #28 Loss: 0.6521810536381557
iter #29 Loss: 0.650796360662

In [12]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

user_factors.weight tensor([[ 1.3681,  1.7746,  1.4515,  ...,  1.0415,  0.4387,  1.2938],
        [ 0.2849,  1.1762,  0.7791,  ...,  1.3439,  0.5829,  1.6879],
        [ 0.8045,  0.6557,  0.5308,  ...,  2.6208, -0.0749, -1.1938],
        ...,
        [ 0.2959,  0.2069,  2.4281,  ...,  1.9627,  1.3915,  0.4520],
        [ 0.9426,  1.4666,  0.6515,  ...,  0.4826,  0.7308,  1.4368],
        [ 1.6665,  1.6248,  0.4369,  ...,  1.0405,  1.3512,  1.1120]],
       device='cuda:0')
item_factors.weight tensor([[ 0.6313,  0.3261,  0.3320,  ...,  0.2521,  0.5448,  0.4496],
        [-0.0530,  0.9863, -0.0323,  ...,  0.5007,  0.6890,  0.6223],
        [ 0.6519,  0.4110,  0.3099,  ...,  0.4513,  0.7211,  0.5324],
        ...,
        [ 0.3596,  0.3576,  0.3318,  ...,  0.3602,  0.3740,  0.3545],
        [ 0.3905,  0.3781,  0.3859,  ...,  0.3868,  0.4019,  0.3797],
        [ 0.4040,  0.3850,  0.3839,  ...,  0.3930,  0.3860,  0.3805]],
       device='cuda:0')


In [13]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [14]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [15]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [16]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also note that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
    print("Cluster #{}".format(cluster))
    movs = []
    # Find movie indices belonging to the current cluster
    for movidx in np.where(kmeans.labels_ == cluster)[0]:
        movid = train_set.idx2movieid[movidx]
        # Check how many ratings this movie has
        rat_count = len(ratings_df.loc[ratings_df['movieId'] == movid])
        movs.append((movie_names[movid], rat_count))
    # Sort movies by rating count in descending order and print top 10
    for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
        print("\t", mov[0])

Cluster #0
	 GoldenEye (1995)
	 Home Alone (1990)
	 Cliffhanger (1993)
	 Back to the Future Part III (1990)
	 Back to the Future Part II (1989)
	 Broken Arrow (1996)
	 Nutty Professor, The (1996)
	 Liar Liar (1997)
	 Pirates of the Caribbean: Dead Man's Chest (2006)
	 American President, The (1995)
Cluster #1
	 Forrest Gump (1994)
	 Shawshank Redemption, The (1994)
	 Silence of the Lambs, The (1991)
	 Braveheart (1995)
	 Fight Club (1999)
	 Seven (a.k.a. Se7en) (1995)
	 Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
	 Fugitive, The (1993)
	 Saving Private Ryan (1998)
	 Sixth Sense, The (1999)
Cluster #2
	 Pulp Fiction (1994)
	 Matrix, The (1999)
	 Star Wars: Episode IV - A New Hope (1977)
	 Terminator 2: Judgment Day (1991)
	 Star Wars: Episode V - The Empire Strikes Back (1980)
	 Lord of the Rings: The Fellowship of the Ring, The (2001)
	 Star Wars: Episode VI - Return of the Jedi (1983)
	 Godfather, The (1972)
	 Batman (1989)
	 Lord of the Rings: The T